# Goal: simulate market dataset such that we can correctly estimate coefficients later on 



In [45]:
import import_ipynb
import importlib
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 200
n_product = 7
n_consumer = 1000
# market_variance = 0.01 #amount of variation between markets 
# endogenous_noise = 0.01 #how noise to generate price 

#structural parameters 
true_alpha = 2.5 # utility decreases with price 
true_beta = 4.7 # utility inceases with characteristics
true_sigma = 0.01 #variance of sensitivity to characteristics

import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)

importlib.reload(f)

changed third


ModuleSpec(name='BLP_functions', loader=<import_ipynb.NotebookLoader object at 0x10bcd41a0>, origin='BLP_functions.ipynb')

In [46]:
all_markets = []
core_char = [1.2, 0.5, -0.8, 0.3, 1.0, -0.6, 0.9]

for market in range(n_market):
    # prod_char = np.random.uniform(-3,3,n_product)
    # prod_char = (prod_char - np.mean(prod_char)) / np.std(prod_char)
    prod_char = core_char #np.random.normal(0, 0.2, n_product) #generate product characteristics
    prod_char = (prod_char - np.mean(prod_char)) / np.std(prod_char)
    # generate market shares and price variables endogenously 
    cost_shifter = np.random.normal(0,2,n_product) #generate cost shifters
    price = f.price_gen(prod_char, cost_shifter) #generate price from characteristics and cost shifters
    price = (price - np.mean(price)) / np.std(price) #normalize price
    inner_utility = f.utility_gen(np.array([true_alpha, true_beta]), true_sigma, prod_char, price, n_consumer) 
    np.clip(inner_utility, -700, 700)
    # print(inner_utility.shape)
    # print("inner_utility", inner_utility)
    inner_prob_choice = f.choice_prob(np.array(inner_utility))
    # print("prob_choice", inner_prob_choice)
    # generate market share directly from the choice probabilities
    outer_market_share = inner_prob_choice.sum(axis=1)/inner_prob_choice.shape[1]
    # print("share", outer_market_share)
    # print(inner_prob_choice.shape)
    # generate the instrument now: calculate and normalize them 
    instrument_cost = np.random.normal(0,2,n_product)
    instrument_cost = (instrument_cost - np.mean(instrument_cost)) / np.std(instrument_cost)
    instrument_price = sum(price) - price
    instrument_price = (instrument_price - np.mean(instrument_price)) / np.std(instrument_price)
    instrument_char = sum(prod_char) - prod_char
    instrument_char = (instrument_char - np.mean(instrument_char)) / np.std(instrument_char)

    market_j = pd.DataFrame({
        'market_id': market,
        'product_id': list(range(n_product)),
        'prod_char': prod_char, 
        'price': price,
        'share': outer_market_share, 
        'true_delta': true_beta * prod_char - true_alpha * price,
        'share_not_buy': 1 - sum(outer_market_share),
        'instrument_cost': instrument_cost,
        'instrument_char': instrument_char,
        'instrument_price': instrument_price
    })
    # print(market)
    all_markets.append(market_j)

print(all_markets[:50])

# print('true_beta', true_beta,'true_alpha', true_alpha, 'true_sigma', true_sigma)
export_dataset = pd.concat(all_markets, ignore_index=True)
export_dataset.to_csv('simulated_market_data.csv', index=False)

# print(np.unique(export_dataset['market_id']).size) #number of markets
# print(np.sum(export_dataset['market_id'] == 0)) #number of product

[   market_id  product_id  prod_char    price   share  true_delta  \
0          0           0    1.15976  0.98629 0.18345     2.98517   
1          0           1    0.19657 -0.52591 0.08694     2.23865   
2          0           2   -1.59222 -1.53879 0.00024    -3.63645   
3          0           3   -0.07863 -0.00124 0.00642    -0.36646   
4          0           4    0.88457 -0.07219 0.70959     4.33794   
5          0           5   -1.31702 -0.58903 0.00008    -4.71743   
6          0           6    0.74697  1.74086 0.00400    -0.84141   

   share_not_buy  instrument_cost  instrument_char  instrument_price  
0        0.00927          1.02951         -1.15976          -0.98629  
1        0.00927          0.18846         -0.19657           0.52591  
2        0.00927          1.04500          1.59222           1.53879  
3        0.00927         -0.93451          0.07863           0.00124  
4        0.00927         -1.72552         -0.88457           0.07219  
5        0.00927          0.